In [3]:
import logging, os
import random

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from matplotlib import colors
import astropy.io.ascii as at
from astropy.io import fits

In [4]:
def chips(filename):

    
    hdu = fits.open(filename)
    header = hdu[0].header
    kpmag = header["KEPMAG"]
    
    if (kpmag>=9.0):
        module = header["MODULE"]
        output = header["OUTPUT"]
    else:
        module, output = -9,-9
    
    return module, output, kpmag

In [5]:
pwd

u'/home/stephanie/code/python/k2phot'

In [6]:
tpfs = at.read("c4_tpfs_box.csv")

In [7]:
modules = np.ones(len(tpfs),int)*-9
outputs = np.ones(len(tpfs),int)*-9
kpmags = np.ones(len(tpfs),float)*-9

for i,fname in enumerate(tpfs["filename"]):
    fname = fname.replace("Dropbox","data")
    if os.path.exists(fname)==True:
        modules[i], outputs[i], kpmags[i] = chips(fname)

In [8]:
keep = np.where(modules>=0)[0]
modules = modules[keep]
outputs = outputs[keep]
kpmags = kpmags[keep]

keep2 = np.argsort(modules)
modules = modules[keep2]
outputs = outputs[keep2]
kpmags = kpmags[keep2]

In [9]:
pairs = zip(modules,outputs)

In [10]:
unique_pairs = [(2,1),(2,2),(2,3),(2,4),
                (4,1),(4,2),(4,3),(4,4),
                (6,1),(6,2),(6,3),(6,4),
                (8,1),(8,2),(8,3),
                (9,2),(9,3),(10,3),(10,4),
                (11,1),(11,3),(12,1),(12,2),
                (13,2),(13,4),(14,1),
                (15,1),(15,4),(16,1),(16,2),
                (17,1),(18,1),(18,3),(18,4),
                (19,1),(20,2)]

In [11]:
len(unique_pairs)

36

In [12]:
all_file = "k2_c4_kp9-16_stars.csv"
all_k2 = at.read(all_file,data_start=2)

In [13]:
print(all_k2.dtype)

[('K2 ID', '<i8'), ('Campaign', '<i8'), ('Object type', 'S4'), ('RA (J2000)', 'S12'), ('Dec (J2000)', 'S12'), ('Target Type', 'S2'), ('Investigation ID', 'S111'), ('KEP Mag', '<f8'), ('Kep Flag', 'S3'), ('Module', '<i8'), ('Output', '<i8'), ('Channel', '<i8')]


In [14]:
n_per_chip = 100
test_epics = np.ones(len(unique_pairs)*n_per_chip,"int32")*-9999
test_mods = np.ones(len(unique_pairs)*n_per_chip,"int")*-9
test_outs = np.ones(len(unique_pairs)*n_per_chip,"int")*-9
test_kps = np.ones(len(unique_pairs)*n_per_chip,"float")*-99.0
for i,p in enumerate(unique_pairs):
    mod,out = p
    this_chip = np.where((all_k2["Module"]==mod) &
                         (all_k2["Output"]==out))[0]
    
    random_on_this_chip = random.sample(this_chip,n_per_chip)
    
    print(mod,out,len(this_chip),len(np.unique(random_on_this_chip)))
    test_epics[i*100:(i+1)*100] = all_k2["K2 ID"][random_on_this_chip]
    test_mods[i*100:(i+1)*100] = mod
    test_outs[i*100:(i+1)*100] = out
    test_kps[i*100:(i+1)*100] = all_k2["KEP Mag"][random_on_this_chip]

(2, 1, 152, 100)
(2, 2, 176, 100)
(2, 3, 150, 100)
(2, 4, 170, 100)
(4, 1, 169, 100)
(4, 2, 170, 100)
(4, 3, 140, 100)
(4, 4, 184, 100)
(6, 1, 117, 100)
(6, 2, 133, 100)
(6, 3, 159, 100)
(6, 4, 132, 100)
(8, 1, 154, 100)
(8, 2, 139, 100)
(8, 3, 162, 100)
(9, 2, 154, 100)
(9, 3, 144, 100)
(10, 3, 242, 100)
(10, 4, 196, 100)
(11, 1, 129, 100)
(11, 3, 127, 100)
(12, 1, 153, 100)
(12, 2, 169, 100)
(13, 2, 173, 100)
(13, 4, 138, 100)
(14, 1, 163, 100)
(15, 1, 318, 100)
(15, 4, 338, 100)
(16, 1, 134, 100)
(16, 2, 144, 100)
(17, 1, 111, 100)
(18, 1, 145, 100)
(18, 3, 132, 100)
(18, 4, 131, 100)
(19, 1, 147, 100)
(20, 2, 171, 100)


In [15]:
print(len(test_epics),len(np.unique(test_epics)))
print(len(np.where(test_epics<0)[0]))

(3600, 3600)
0


In [16]:
f = open("k2_c4_kp9-16_stars_tpf_download.lst","w")
g = open("k2_c4_kp9-16_stars_tpf_download01.sh","w")

ep_filenames = np.empty(len(test_epics),"S50")
ep_filenames[:] = ""

for i, epic in enumerate(test_epics):
    epic = str(epic)
    ep0 = epic[:4]
    ep1 = epic[4:6]
    f.write("http://archive.stsci.edu/pub/k2/target_pixel_files/c4/")
    tpf_file = "{0}00000/{1}000/ktwo{2}-c04_lpd-targ.fits.gz".format(ep0,ep1,epic)
    ep_filenames[i] = tpf_file
    f.write("{0}\n".format(tpf_file))
    if i<100:
        g.write("wget -q http://archive.stsci.edu/pub/k2/target_pixel_files/c4/")
        g.write("{0}\n".format(tpf_file))

f.close()

In [17]:
data = {"filename":ep_filenames, "EPIC":test_epics, "Module":test_mods, "Output":test_outs, "Kp":test_kps}
at.write(data,"k2_c4_kp9-16_stars_tpfs.csv",names = ["filename","EPIC","Module","Output","Kp"])